# Training: SQL (Heavy-users) 💪
Welcome to the training notebook on using SQL.


This notebook is pitched at heavy users who perform data management roles to store objects permanently in SQL. 

They will have **READ-WRITE** or higher access to the database.

![SQL girl meme](https://live.staticflickr.com/65535/49198404911_55085b0307_z.jpg "SQL girl meme") 

# What will this session cover? 👂
This session will show you how to do the following things in SQL:
1. Dynamic SQL querying
1. What makes a well-governed database?
1. Creating and updating tables
1. Importing data into SQL
1. Indexing columns to improve querying speeds
1. Adding constraints to columns to restrict entries that can go inside it
1. Using stored procedures and functions to do more bespoke operations
1. Database triggers for tracking activity
1. Version-controlling databases

In [2]:
-- set database to use
USE [AdventureWorks];

# 1. Dynamic SQL querying 🤸‍♂️
This is essentially SQL code that writes SQL code. It's particularly useful when you want to parameterise your SQL statements with an input variable. 

To be able to employ dynamic SQL querying, you'll need to know how to:
1. Create a variable in SQL and assign it a value
1. Create a SQL query which takes the variable you created
1. Execute the SQL query

The core idea behind dynamic SQL querying lies in being able to write your query as a string/varchar/text, and then execute/run the text as if it is a SQL query.

> **USER STORY:** *As a lazy-ass, punk-ass, funk-ass SQL version of NAS, I want to quickly run a SQL statement to quickly count all the number of rows in several tables in my database rather than write the same* `SELECT COUNT(*) FROM <*table_name*>` *statement several times but with different table names, so I can get back to making my new coding mixtape.* 🎤🎧

The code below is a rubbish example of the power of dynamic SQL querying. A better illustration of when and why you woukd prefer to use dynamic SQL querying can be found in the exercises in *(4.)*.



In [3]:
-- create and set variable/parameters
DECLARE @name_schema AS NVARCHAR(25) = 'Production'
DECLARE @name_table AS NVARCHAR(50) = 'Location'

-- write dynamic SQL as string/text
DECLARE @query AS NVARCHAR(MAX) = 
'
    SELECT COUNT(*)
    FROM [' + @name_schema + '].[' + @name_table + '];
' 

-- print query to see what SQL is written
PRINT @query

-- excute dynamic SQL being written as string/text
EXEC sp_executesql @query

## 🐴 EXERCISE: Dynamic SQL querying #1
**Question:** Is there an even more efficient way to perform the same task in (1.) without having to use dynamic SQL querying?

**Hint:** Consider using System Tables belonging to the `[sys]` database.

**Note:** This exercise introduces you to some very useful tables that exist in the background which will be extremely useful for data management tasks. You may also explore the tables in the `[INFORMATION_SCHEMA]` database. Lastly, it shows that dynamic SQL is not always the best option. 

In [0]:
-- please write your answer below

# 2. What makes a well-governed database? 📣
The attitude to storing data and code in SQL should be different to storing data and code in a folder. As a database administrator/architect/manager, your role is to effectively govern the storage and access of data so that it can be intuitively found, easily accessed and fluidly recycled/reused.

This means that what is held in SQL should meet the following criteria:

- [ ] Be **C**onsistent - all naming should follow a style-guide so that users of the databse can navigate easily to where they want to go.
    - Within this, you also want to explicitly and distinctly separate out the various objects by giving a specific naming to tables, Views, stored procedures etc.
    - Such a style-guide is available [here](https://github.com/avisionh/Training-SQL/wiki/Style-Guide:-T-SQL)
- [ ] Be **O**pen - in a database that's accessible to a wide range of people, create with their perspective in mind. 

> **Example I:**
> - `[usr].[Y14_15_Student_Fix_Core]`
> - `[usr].[Y14_15_Student_Fix_Fix_Core]`
> - `[usr].[Y14_15_Student_Fix_Old_Core]`
> - This is horrendous! What the heck does *Fix_Fix*, *Fix_Old* mean? Blimey!!!

- [ ] Be **P**ersistent - temporary or test objects should be wiped. Only objects that are useful should be there.
    - If a temporary or test object is needed later, consider if it can be made into a permanent table, a temp table or stored in a database specifically for experimentation.

> **Example II:** `[dbo].[SLC_extract_140917_temp]` is horrific! Having several of them is even worse.

- [ ] Be **O**rderly - make use of naming to order objects appropriately

> **Example III:** Should you organise the below four tables by collection or year?
> - `[usr].[Y14_15_Staff_Core]`
> - `[usr].[Y14_15_Student_Core]`
> - `[usr].[Y15_15_Staff_Core]`
> - `[usr].[Y15_16_Student_Core]` 


- [ ] Be **U**nderstanding - you have to balance out each team's considerations and ensure they all agree to a common framework/standard
    - How do you tackle the issue presented in *Example I*?
- [ ] Be **T**hematic - consider what is themed and bring them together
    - Do you have many tables that do lookups? Why not put them together into a general structure?

If you like pneumonics, this means you should be a **COP OUT**.

![SQL roll safe meme](https://i.imgur.com/Hya1F1s.gif "SQL roll safe meme") 


# 3. Creating and updating tables ♻️
When deciding that **temporary tables** are unsuitable for your usage because you want to increase access to the data explicitly for more people or people will be building off your query to view the **CTE**, then creating and updating a table to store permanently in a SQL database is appropriate. 

When creating a table, you may want to store it in **tidy data format** (covered in *Training: SQL (Medium-users))*) so the same data but in a different version or time can be imported to this, meaning you are **updating** the ewly-created table.

To create a table, you need to know a few things upfront such as the columns it will contain, their data types and version. 



``` SQL
-- template: create table query
CREATE TABLE [<schema_name>].[<table_name>] (
    [Column_1] <datatype>
    ,[Column_2] <datatype>
    ,...
);

-- template: update table 
UPDATE [<schema_name>].[<table_name>]
SET [column_n] = <new_value>
WHERE [column_j] = <filter_value>;
```

# 4. Importing data into SQL 🏳️‍🌈
After deciding that you want to store the data in a table on SQL, and having creatied the tables, you next job is to actually import the data into the table. There are several methods.
1. **SQL Server Import Wizard:** Good for relatively quick, one-time imports of datasets. Bad for multiple table imports of the same datasets.
1. **Powershell:** Fantastic for bulk-inserting multiple datasets. Bad if you don't want to learn another coding/scripting language.
1. **Bulk load via SQL code:** Good for importing multiple datasets directly into SQL. Bad if you don't have high permission rights. 

```
-- table to loop through file names 
CREATE TABLE [AdminDetails].[SourceData]
(
    [PathFile] NVARCHAR(MAX)
    ,[NameFile] NVARCHAR(MAX)
)

-- declare variables
DECLARE @filename AS NVARCHAR(250)
    ,@filepath AS NVARCHAR(MAX)
    ,@codesql AS NVARCHAR(MAX)
    ,@command AS NVARCHAR(1000)

-- get list of files to process
SET @filepath = 'C:\Temp\'
SET @command = 'dir ' + @path + '*.csv /b'

INSERT INTO [AdminDetails].[SourceData]([NameFile])
EXEC Master..xp_cmdShell @command

UPDATE [AdminDetails].[SourceData] 
SET [PathFile] = @filepath 
WHERE [PathFile] IS NULL

-- cursor loop
DECLARE c1 CURSOR FOR
    SELECT [PathFile]
        ,[NameFile]
    FROM [AdminDetails].[SourceData]
    WHERE [PathFile] LIKE '%.csv%'

OPEN c1

FETCH NEXT FROM c1 INTO @filepath, @filename
WHILE @@fetch_status != -1
    BEGIN
        -- bulk insert won't take a variable name, so make a parameter for SQL to read and execute
        SET @codesql = 
        '
            BULK INSERT [<schema_name>].[<table_name>]
            FROM ''' + @filepath + @filename + ''' ' + '
            WITH(
                FIELDTERMINATOR = '',''
                ,ROWTERMINATOR = ''\n''
                ,FIRSTROW = 2
            ) '
PRINT @codesql
EXEC (@codesql)
```

*Code slightly adapted from [here](https://stackoverflow.com/questions/16076309/import-multiple-csv-files-to-sql-server-from-a-folder)*

1. **Direct insertion via SQL code:** Good for realtively quick, one time imports of rows. Bad for anything outside of this.
```
INSERT INTO [<schema_name>].[<table_name>] (
    [column_name_1]
    ,[column_Name_2]
    , ...
    , [column_name_n]
)
VALUES
    (row_1_value_1, row_1_value_2, ..., row_1_value_n)
    ,(row_2_value_1, row_2_value_2, ..., row_2_value_n)
    ,...
    ,(row_k_value_1, row_k_value_2, ..., row_k_value_n)
```
1. **SSIS Package:** Good fdoing more bespoke things like not inserting duplicate rows. Bad for bulk importing multiple datasets.
1. **R:** Good as this is a relatively easy programming language to learn. Bad if you are trying to import a large amount of data.

We define *large* in point (4.) with regards to the size of the ***data being imported into SQL being larger than the amount of RAM your computer has.*** This is because importing data into SQL from R requires importing the data into your R session first, then moving it across into SQL. As R stores data on the computers RAM for fast retrieval (relative to storage on a hard disk), then if the data was larger than the RAM, this would be a very slow process. 

## 🍾 EXERCISE: Inserting data into tables
In the below code, we have created a table for you to work with.
```
CREATE TABLE [Person].[ProductPreference]
(
   [StaffId] INT IDENTITY(1,1) PRIMARY KEY
   ,[StaffUnit] NVARCHAR(25)
   ,[BookFavourite] NVARCHAR(100)
   ,[BookAuthor] NVARCHAR(100)
   ,[SongFavourite] NVARCHAR(100)
   ,[SongArtist] NVARCHAR(100)
   ,[FilmFavourite] NVARCHAR(100)
   ,[FilmDirector] NVARCHAR(100)
   ,[DataTimeCreation] DATETIME DEFAULT(GETDATE()) 
)
```
**QUESTION:** Can you complete one row of this table with your preferences?

**NOTE:** From the definitions of the `[StaffId]` and `[DateTimeCreation]` fields, there is no need for you to enter these yourselves. Do you know what they do?

In [0]:
-- please write your answer below

## 🙆‍♀️ EXERCISE: Thinking about objects in SQL
**QUESTION:** Consider the following discussion questions below:
1. Thinking about the `[StaffId]` column, do you know what it does? 
1. Why might the name of this column be inappropriate for what it actually does? 
1. Can you think of a way to improve this?

In [0]:
-- please write your answers below

## 🤯 EXERCISE: Dynamic unpivoting
In actual fact, the table created above, `[Person].[ProductPreference]` is not in a tidy data form. As a database manager, this is a personal affront to you as you want to provide analysts with a formatted table of data that makes their jobs easier.

**QUESTION:** Can you unpivot the `[Person].[ProductPreference]` table so that it is in **tidy data** format? Try to unpivot it without explicitly referncing the columns you want to unpivot to rows.

**NOTE:** This exercise is designed to get you thinking about what makes a good, structured table in SQL that's helpful to the end-user, the analyst. It brings in concepts covered in *(1.)* and *(2.)*. It also shows you the power of dynamic querying.

In [0]:
-- please write your answer below

# 5. Indexing columns to improve querying speeds 📗
With really big tables, it makes sense to **index** their columns so that it is much more faster to query from them via filtering/joining or other SQL operations.

**Column indexes/indices** are essentially a way of creating "bookmarks" in your data so that when you're filtering from it for instance, the SQL engine looks through the bookmarks and filters on them, rather than go into each entry, row-by-row, and filtering.

They do slow down `UPDATE` and `INSERT` operations so you may want to drop **indexes** before performing these operations and then reapply the **indexes**.

There are two types of **indexes**:
- 📫 **Clustered:** Physically orders the data on the ~~hard~~ disk. This can make `ORDER BY` operations significantly faster.
    + Only one can be created per table.
    + Faster to read than non-clustered index as data is physically stored in an index order.
    + Does not take additional memory to store.
- 📬 **Non-clustered:** Defines a "logical" order that does not match the physical order on the ~~hard~~ disk. This "logical" order is like a layer on top of a physical order, where a pointer is used.
    + More than one can be created per table as they can be applied to columns.
    + Data insertion/update is faster than clustered index. 
    + Needs additional memory to store.

![Clustered image](https://i.stack.imgur.com/kFSWR.png "Clustered image")

> **TIP:** Typically, **clustered indexes** are created on the most unique column or columns in your data, such as a unique identifier (also known in SQL parlance as the **primary key**).
>> For the most part, we use **non-clustered indexes** on columns which we plan to filter by and join on and drop them before inserting or updating data in the tables.

```
-- template: create index
CREATE NONCLUSTERED INDEX IX_<table_name>_<column_name>
    ON [<schema_name>].[<table_name>] ([<column_name>])
    -- enable faster index creation/rebuild times
    -- without this, can dramaticall increase tempdb file sizes, which is bad!
    WITH (SORT_IN_TEMPDB = ON);
```

```
-- template: drop multiple indexes, checking if they exist first
IF EXISTS 
(
    SELECT [name] FROM [sys].[indexes]
    WHERE [name] IN (N'IX_<table_name>_<column_name_1>', 'IX_<table_name>_<column_name_2>', ...)
) 
BEGIN
    DROP INDEX IX_<table_name>_<column_name_1>
        ON [<schema_name>].[<table_name>]
    DROP INDEX IX_<table_name>_<column_name_2>
         ON [<schema_name>].[<table_name>]
    ...
END

-- need to specify what to do if these indexes don't exist
-- otherwise SSIS package will throw error because won't know
-- what to do if indexes don't exist
ELSE 
BEGIN
    WAITFOR DELAY '00:00:00'
END;
```

# 6. Adding constraints to columns 💸
Adding **constraints** to columns in you data ensures the integrity of possible values being entered into the table. They specify rules that data must adhere to in your tables.

Specifically of interest is the **check constraint** which sets a specified list of values that the column can take, and any data being entered into the table that does not fulfill this pre-specified list of values cannot be imported until this is resolved.

```
-- template: create constraint on existing table
ALTER TABLE [<schema_name>].[<table_name>]
ADD CONSTRAINT CHK_<constraint_name> CHECK ([<column_integer>] > n AND [<column_string>] IN ('string_1', 'string_2', ...))
```

```
-- template: drop constrain on existing table
ALTER TABLE [<schema_name>].[<table_name>]
DROP CHECK CHK_<constraint_name>
```

## 👥 EXERCISE: Adding contraint to prevent rogue entries
**QUESTION:** In the `[Person].[ProductPreference]` table, can you add a constraint on the `[FilmFavourite]` column to ensure that is must always be completed?

In [0]:
-- please write your answer below

# 7. Using stored procedures and functions 🛠🔦
**Stored procedures** and **functions** both execute a set of SQL instructions to return a result. However they are different in subtle ways.
***
## 7.i. Stored Procedures 🗜
A **Stored procedures** is a group of SQL statements stored in the database. They can be called from SQL, another **stored procedure**, the command line, or a different program such as R or Excel.

For instructions on how to create **stored procedures** within SSMS, see the official documentation [here](https://docs.microsoft.com/en-us/sql/relational-databases/stored-procedures/create-a-stored-procedure?view=sql-server-ver15).

Advantages | Disadvantages
--- | --- |
Can be set-up to run automatically on a schedule | Harder to version-control unless you use method in *(9.)*
Can **resuse** code many times | Debugging can be slightly harder if not designed well
Held centrally alongside data, so clearer governance | `NULL`

### Batching 🍫
Regarding designing a **stored procedure** well, there are several useful commands you can use to create a robust **stored procedure** that is easier to follow and catches errors. Idea is that you can write and run your **stored procedure** in one go, but it is more helpful to run it in batches, so you can also include helpful messages that signal the progress of the **stored procedure**.
- Think of running your **stored procedure** in batches as equivalent to highlighting your code in chunks and running them separately.

You set-up your batches as follows:
1. `BEGIN TRANSACTION` - signals to SQL server the start point of a batch of T-SQL statements
1. `GO` - signals to SQL server to run a set of T-SQL statements
1. `COMMIT` - signals to SQL server to make permanent changes in a table based on a batch of code
1. `ROLLBACK` - signals to SQL to cancel **all** changes previously executed in a batch of code

### Error trapping 💔
Using the `TRY ... CATCH` statement, you can specify what happens when you capture and output errors. The structure of it is as follows:
```
BEGIN TRY
    <sql_statement>
END TRY
BEGIN CATCH
    PRINT ERROR_MESSAGE()
    PRINT ERROR_NUMBER()
END CATCH
GO
```

In [2]:
-- error trapping
BEGIN TRY
    -- generate a divide-by-zero error
    SELECT 1/0;
END TRY
BEGIN CATCH
    SELECT 
        [ErrorNumber] = ERROR_NUMBER()
        ,[ErrorSeverity] = ERROR_SEVERITY()
        ,[ErrorState] = ERROR_STATE()
        ,[ErrorProcedure] = ERROR_PROCEDURE()
        ,[ErrorLine] = ERROR_LINE()
        ,[ErrorMessage] = ERROR_MESSAGE()
END CATCH;
GO

### Combining running batches with error trapping 🐀
By combining running batches and error trapping in your **stored procedure** definition, then you are able to create a well-documented, easy-to-follow, robust **stored procedure**. You can avoid saving what happened and instead, move the whole of the `TRY` section of the query back through using `ROLLBACK`, so long as you have not used a `COMMIT` statement.
```
BEGIN TRY
    <sql_statement_one>
    <sql_statement_two>
    ...
END TRY
BEGIN CATCH
    IF @@TRANCOUNT >= 1
    ROLLBACK
    ... <error_logging>
END CATCH;
GO
```

Typically, it is quite difficult to write a **stored procedure** from first principles or even a template. A better approach is to find an existing **stored procedure** and plagiarise it! Not only does it save you brain power, but it is faster and more error-proof.

In [1]:
-- create stored procedure by bringing everything together
CREATE PROCEDURE [Person].[spr_InsertPersonalDetails]
    -- add parameters to pass into stored proc here
    @FirstName AS NVARCHAR(50)
    ,@LastName AS NVARCHAR(50)
    ,@Age AS SMALLINT
    ,@Active AS BIT
    ,@Salary AS MONEY
    ,@PPFDeduction AS MONEY
AS
BEGIN TRANSACTION
    BEGIN TRY
        -- insert into [PersoanlDetails] table
        INSERT INTO [PersonalDetails] ([FirstName], [LastName], [Age], [Active])
        VALUES (@FirstName, @LastName, @Age, @Active)
    GO
        DECLARE @pdId AS INT
        SET @pdId = SCOPE_IDENTITY()
        -- insert into [Accounts] table
        INSERT INTO [Accounts] ([Salary], [PPFDeduction], [PersonalDetailsId])
        VALUES(@Salary, @PPFDeduction, @pdId)
        -- if not error, commit the transaction
    COMMIT TRANSACTION
    END TRY

    BEGIN CATCH
    -- if error, rollback any changes done by any of the SQL statements
        ROLLBACK TRANSACTION
    END CATCH


## 7.2. Functions 🌂
**Functions:** subprograms commonly used and reused throughout SQL database application for cleaning and manipulating.

Even humble aggregation functions such as `SUM(), AGGREGATE(), ...)` are **functions**.

They are good for the following things:
- Are written once but used multiple times - saving time and effort in supporting modular programme too. 
- Improve performance and efficiency of the database.
- Complex progaming logic can be decompoased into a smaller and simpler functions, making it easier to understand and maintain. 

### Difference between stored procedures and functions 🍊🍎
Whilst both sound very similar in their operations and constructions, there are subtle differecne between them:

Stored Procedures | Functions
--- | --- |
Does not return a value, just `0` or `n` values | Always returns a value
Can have input **and** output parameters | Only have input parameters for it
Cannot call them from **Functions** | Can call them from **Stored Procedures**
Allows `SELECT/INSERT/UPDATE/DELETE` statements | Only allows `SELECT` statement
`TRY-CATCH` block can handle exceptions | `TRY-CATCH` cannot be used
Cannot be embedded in `WHERE/HAVING` | Can be embedded in `WHERE/HAVING`
Returned tables cannot be used in `JOIN` | Retuned tables treated as another rowset and can `JOIN`

In [0]:
-- create a simple stored proc
CREATE PROCEDURE spr_HelloWorld
AS
PRINT 'Hello World'

-- execute stored proc
EXEC spr_HelloWorld

In [0]:
-- create a simple function
CREATE FUNCTION dbo.fn_helloworld()
RETURNS varchar(20)
AS 
BEGIN
	 RETURN 'Hello world'
END;

GO;

-- call the function
SELECT dbo.fn_helloworld()

## 🤼‍♀️ EXERCISE: Stored procedures and functions
**QUESTION:** From running the stored procedure and function defined above, what is the difference in how they are prompted to run?

In [0]:
-- please write your answer below

# 8. Database triggers to record activity 📹📸📼
**Database triggers** are extremely useful in automatically recording acitivty of permanenet changes to a database. It can captures things like:
- What the change was
- Who made the change
- When the change was made
- Where the change was made in regards to the objects affected
- How the change was made (displays the code)

As for *Why the change was made*, that's a question that is more easily answered now that you can easily see who made the change and when - it helps you identify the right person to find out, rather than before when you had to scrabble around looking for the right person.

At a basic level, it can also offer a rudimentary form of **version-controlling** from the way the code that caused the change can also be captured. 

In [9]:
-- create schema to assign database activity table in
IF NOT EXISTS
(
	SELECT [SCHEMA_NAME]
	FROM [INFORMATION_SCHEMA].[SCHEMATA]
	WHERE [SCHEMA_NAME] = 'AuditDetails'
)

BEGIN
EXEC sp_executesql N'CREATE SCHEMA [AuditDetails] AUTHORIZATION [dbo]'
END;

In [0]:
-- create table to record database activity
CREATE TABLE [AuditDetails].[DatabaseChangeLog](
	[ChangeId] [int] IDENTITY(1,1) NOT NULL
	,[ChangeDate] [datetime] NOT NULL 
		CONSTRAINT [DF_ddl_log_ChangeDate]
			DEFAULT (GETDATE())
	,[NameUser] [nvarchar](50) NOT NULL
		CONSTRAINT [DF_ddl_log_NameUser]   
            DEFAULT (CONVERT([nvarchar](50), USER_NAME(), (0)))
	,[NameSecurity] [nvarchar](50) NOT NULL
		CONSTRAINT [DF_DDLChangeLog_NameSecurity]   
            DEFAULT (CONVERT([nvarchar](50), SUSER_SNAME(), (0)))
	,[NameLogin] [nvarchar](50) NOT NULL
		CONSTRAINT [DF_DDLChangeLog_NameLogin]   
            DEFAULT (CONVERT([nvarchar](50), original_login(),(0)))
	,[EventType] [nvarchar](100) NULL
	,[ObjectName] [nvarchar](100) NULL
	,[ObjectType] [nvarchar](100) NULL
	,[TsqlCode] [nvarchar](max) NULL
) ON [PRIMARY] TEXTIMAGE_ON [PRIMARY];

In [0]:
-- create and turn on database trigger
CREATE TRIGGER trg_DatabaseChangeLog ON DATABASE 
    FOR DDL_DATABASE_LEVEL_EVENTS 
AS 
    DECLARE @data XML 
    SET @data = EVENTDATA() 
    IF @data.value('(/EVENT_INSTANCE/EventType)[1]', 'nvarchar(100)') <> 'CREATE_STATISTICS'  
		INSERT INTO [AuditDetails].[DatabaseChangeLog] 
		( 
			[EventType], 
			[ObjectName], 
			[ObjectType], 
			[TsqlCode]
		) 
		VALUES  
		( 
			@data.value('(/EVENT_INSTANCE/EventType)[1]', 'nvarchar(100)'), 
			@data.value('(/EVENT_INSTANCE/ObjectName)[1]', 'nvarchar(100)'), 
			@data.value('(/EVENT_INSTANCE/ObjectType)[1]', 'nvarchar(100)'), 
			@data.value('(/EVENT_INSTANCE/TSQLCommand)[1]', 'nvarchar(max)') 
		);

-- turn-on trigger
ENABLE TRIGGER [trg_DatabaseChangeLog] ON DATABASE;

In [0]:
-- create View to pull in schema name so can better identify tables
CREATE VIEW [AuditDetails].[vw_DatabaseChangeLog]
AS 
(
    SELECT changelog.[ChangeId]
        ,changelog.[ChangeDate]
        ,changelog.[NameUser]
        ,changelog.[NameSecurity]
        ,changelog.[NameLogin]
        ,changelog.[EventType]
        ,[SchemaId] = admintable.[schema_id]
        ,[SchemaName] = OBJECT_SCHEMA_NAME(admintable.[object_id])
        ,[ObjectId] = admintable.[object_id]
        ,changelog.[ObjectName]
        ,[ObjectType] = admintable.[type_desc]
        ,changelog.[TsqlCode]
    FROM [AuditDetails].[DatabaseChangeLog] AS changelog
    LEFT JOIN [sys].[all_objects] AS admintable
        ON changelog.[objectName] = admintable.[name]
    -- Keep only 'genuine'/real-people SQL users
    WHERE changelog.[NameUser] != 'dbo'
);

In [0]:
-- see what activity has been recorded
SELECT TOP 100 *
FROM [AuditDetails].[vw_DatabaseChangeLog];

## 🔭 EXERCISE: Querying from a big-brother table
**QUESTION:** In the `[AuditDetails].[vw_DatabaseChangeLog]` table, can you query it to look for changes that you have done to the database?

In [0]:
-- please write your answer below

# 9. Version-controlling your database ✨🌈
Originating from the software development world, effectively tracking and commenting what and why a change was made to your code, whilst retaining the ability to easily revert to an old state of your code, **version-controlling** is now a core principle in any general best practice approaches to writing code.

Whereas in *(8.)* offers a rudimentary approach to **version-control** in the way it captures the actual SQL code used to make the change, and has accompaying information on who made the change and when it was made, it does not realise the full vision of **version-control** - namely, it is the ability to revert to older states easily and working collaboratively.

> **PRIOR KNOWLEDGE RQUIRED:** This section assumes a knowledge of Git, especially its terminology.

At the first-level, you can save a copy/script of each object you create in SQL and *git* version-control those scripts, though this can quickly get tedious when you start having around 15 or more tables, Views, stored procedures, functions database triggers, constraints to manage.

Enter the second-level, where through **Visual Studio**, you can easily take a copy of all the objects in your database, save them on to a drive and **version control** via **Git**, ensuring you `push` and `pull()` from the remote repo on GitHub or Azure DevOps.

Additional guidance is available on Github [here](https://github.com/avisionh/SQL-Titbits/wiki/User-Guide:-SQL-x-Git-Version-Control).